# COMP 424 Assignment 1

## Implement Hillclimbing

In [1]:
import numpy as np
import random
import pandas as pd
import copy

def find_max(x, y, function, step = 0.01, lower_bound = 0, upper_bound = 10):
    neighbors = [x - step, y - step, x - step, y + step, x + step, y - step, x + step, y + step, x, y - step, x, y + step, x - step, y, x + step, y]
    maxi = function(x , y)
    coordinates = [x , y]
    for index in range(0, len(neighbors) - 1, 2): 
        result = function(neighbors[index], neighbors[index + 1])
        if result > maxi and not(neighbors[index] < lower_bound or neighbors[index] > upper_bound or neighbors[index + 1] < lower_bound or neighbors[index + 1] > upper_bound):
            maxi = result
            coordinates = [neighbors[index], neighbors[index + 1]]
    return maxi, coordinates

        

## Initialize Functions

In [2]:
def function_one(x, y):
    return np.sin(x / 2) + np.cos(y / 2)

def function_two(x, y):
    return -abs(x - 2) - abs(0.5 * y - 1) + 3

In [3]:
def algorithm(init_x, init_y, function, step_size):
    result_old = find_max(init_x, init_y, function, step_size)
    iteration = 1
    while True:
        result = find_max(result_old[1][0], result_old[1][1], function, step_size)
        if result[1][0] == result_old[1][0] and result[1][1] == result_old[1][1]:
            return {'init_x' : init_x,'init_y': init_y,'max': result[0],'end_x': result[1][0], 'end_y': result[1][1], 'iteration': iteration}
        result_old = result
        iteration += 1
    return None

## Run Algorithm

In [4]:
step = 0.01
results = []
for i in range(0, 100): 
    init_x = random.randint(0 , 10)
    init_y = random.randint(0 , 10)
    result = algorithm(init_x, init_y, function_one, step)
    results.append(result)
frame = pd.DataFrame(data = results)

In [5]:
frame.std()

end_x          1.637367
end_y          4.604610
init_x         3.170827
init_y         3.078370
iteration    164.715665
max            0.577175
dtype: float64

In [6]:
frame.mean()

end_x          3.551600
end_y          3.001600
init_x         4.920000
init_y         4.720000
iteration    371.610000
max            1.667561
dtype: float64

## Local Beam Search Algorithm

In [28]:
def local_beam_search(function, step = 0.01, beam_width = 2, lower_bound = 0, upper_bound = 10):
    coords = []
    for i in range(0, beam_width):
        coords.append({'x' : random.randint(0, 10), 'y' : random.randint(0, 10)})
    while True:
        neighbors = []
        for i in range(0, beam_width):
            neighbors.extend(get_neighbors(coords[i]['x'], coords[i]['y'], function, step, lower_bound, upper_bound))
        
        coords_temp = copy.deepcopy(coords)
        coords.clear()
        max_values = sorted(neighbors, key=lambda k: k['value'], reverse=True) 
        for i in range(0, beam_width):
            coords.append(max_values[i])
        done = True
        for i in range(0, beam_width):
            check = False;
            for j in range(0, beam_width):
                if coords[i]['x'] == coords_temp[j]['x'] and coords[i]['y'] == coords_temp[j]['y']:
                    check = True
                    break
            if not check:
                done = False
                break;
        if done:
            return sorted(coords, key= lambda k: k['value'], reverse=True)[0]
        neighbors.clear()

In [29]:
def get_neighbors(x, y, function, step, lower_bound, upper_bound):
    results = []
    neighbors = [x, y, x - step, y - step, x - step, y + step, x + step, y - step, x + step, y + step, x, y - step, x, y + step, x - step, y, x + step, y]
    for i in range(0, len(neighbors), 2):
        results.append({'x': neighbors[i], 'y': neighbors[i + 1], 'value': function(x, y)})
        
    return results

In [38]:
print(local_beam_search(function_one))

{'x': 0.07000000000008363, 'y': 3.0700000000001264, 'value': 0.07078153712394292}
